In [1]:
from interpret.glassbox import ExplainableBoostingClassifier
import numpy as np
import openml
import joblib
import pandas as pd
import gamchanger as gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from caafe import CAAFEClassifier
import openai
import caafe
import pickle
from caafe import data
from caafe import preprocessing
from interpret import show
from tabpfn.scripts import tabular_metrics
import os


# TODO: Change key before sharing
seed = 0

import t2ebm
import t2ebm.graphs as graphs

In [2]:
X_train = pd.read_csv("spaceship_train.csv")
y_train = X_train["Transported"]
del X_train["Transported"]
del X_train["PassengerId"]


X_test = pd.read_csv("spaceship_test.csv")
del X_test["PassengerId"]
y_test = pd.read_csv("sample_submission.csv")
del y_test["PassengerId"]


"""
ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=1)
ebm.fit(X_train, y_train)
"""

'\nebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=1)\nebm.fit(X_train, y_train)\n'

In [13]:
with open("ebm_spaceship.pkl", "rb") as file:
    ebm = pickle.load(file)

In [5]:
#t2ebm.describe_graph("gpt-4o", ebm, 9)

dataset_description = "In this competition your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system."

y_axis_descripton = """The y-axis depicts contributions in log-odds towards the probability
that the passenger was transported to another dimension."""

t2ebm.describe_graph("gpt-4o",
                     ebm,
                     0,
                     dataset_description=dataset_description,
                     graph_description=y_axis_descripton)

In [7]:
graph_spa = graphs.extract_graph(ebm, 9)
graph_as_text_spa = graphs.graph_to_text(graph_spa, max_tokens=1000)
print(graph_as_text_spa)

INFO: The graph of feature Spa was simplified by 3.0%.
This graph represents a continuous-valued feature. The keys are intervals that represent ranges where the function predicts the same value.

Feature Name: Spa
Feature Type: continuous
Means: {"(0.0, 49.5)": 0.425, "(49.5, 156.5)": 0.236, "(156.5, 240.5)": 0.048, "(240.5, 305.5)": -0.154, "(305.5, 436.5)": -0.346, "(436.5, 452.5)": -0.546, "(452.5, 571.0)": -0.743, "(571.0, 711.5)": -0.932, "(711.5, 930.0)": -1.12, "(930.0, 1343.5)": -1.311, "(1343.5, 1372.5)": -1.676, "(1372.5, 1557.0)": -1.954, "(1557.0, 1870.5)": -2.142, "(1870.5, 2462.5)": -2.358, "(2462.5, 2905.5)": -2.795, "(2905.5, 3214.0)": -3.26, "(3214.0, 4107.0)": -4.308, "(4107.0, 6697.5)": -5.532, "(6697.5, 22408.0)": -5.72}
Lower Bounds (95%-Confidence Interval): {"(0.0, 49.5)": 0.344, "(49.5, 156.5)": 0.166, "(156.5, 240.5)": -0.03, "(240.5, 305.5)": -0.277, "(305.5, 436.5)": -0.498, "(436.5, 452.5)": -1.026, "(452.5, 571.0)": -0.944, "(571.0, 711.5)": -1.052, "(711.5

In [8]:
graph_as_text

'This graph represents a continuous-valued feature. The keys are intervals that represent ranges where the function predicts the same value.\n\nFeature Name: Spa\nFeature Type: continuous\nMeans: {"(0.0, 49.5)": 0.425, "(49.5, 156.5)": 0.236, "(156.5, 240.5)": 0.048, "(240.5, 305.5)": -0.154, "(305.5, 436.5)": -0.346, "(436.5, 452.5)": -0.546, "(452.5, 571.0)": -0.743, "(571.0, 711.5)": -0.932, "(711.5, 930.0)": -1.12, "(930.0, 1343.5)": -1.311, "(1343.5, 1372.5)": -1.676, "(1372.5, 1557.0)": -1.954, "(1557.0, 1870.5)": -2.142, "(1870.5, 2462.5)": -2.358, "(2462.5, 2905.5)": -2.795, "(2905.5, 3214.0)": -3.26, "(3214.0, 4107.0)": -4.308, "(4107.0, 6697.5)": -5.532, "(6697.5, 22408.0)": -5.72}\nLower Bounds (95%-Confidence Interval): {"(0.0, 49.5)": 0.344, "(49.5, 156.5)": 0.166, "(156.5, 240.5)": -0.03, "(240.5, 305.5)": -0.277, "(305.5, 436.5)": -0.498, "(436.5, 452.5)": -1.026, "(452.5, 571.0)": -0.944, "(571.0, 711.5)": -1.052, "(711.5, 930.0)": -2.195, "(930.0, 1343.5)": -1.736, "(1

In [10]:
graph_home = graphs.extract_graph(ebm, 0)
graph_as_text_home = graphs.graph_to_text(graph_home)

In [11]:
graph_as_text_home

'This graph represents categorical feature. Each key represents a possible value that the feature can take.\n\nFeature Name: HomePlanet\nFeature Type: categorical\nMeans: {"Earth": -0.3035, "Europa": 0.5757, "Mars": 0.1064}\nLower Bounds (95%-Confidence Interval): {"Earth": -0.3229, "Europa": 0.5307, "Mars": 0.0669}\nUpper Bounds (95%-Confidence Interval): {"Earth": -0.2842, "Europa": 0.6206, "Mars": 0.146}\n'

In [12]:
print(graph_as_text_home)

This graph represents categorical feature. Each key represents a possible value that the feature can take.

Feature Name: HomePlanet
Feature Type: categorical
Means: {"Earth": -0.3035, "Europa": 0.5757, "Mars": 0.1064}
Lower Bounds (95%-Confidence Interval): {"Earth": -0.3229, "Europa": 0.5307, "Mars": 0.0669}
Upper Bounds (95%-Confidence Interval): {"Earth": -0.2842, "Europa": 0.6206, "Mars": 0.146}

